**Yfinnace version**

The following is a fairly simple Quant. Momentum strategy that can be used to distribute a portfolio between recently high performing stocks

This version, HQ = High Quality, uses various dates, stats, to review longer term safer stocks

In [1]:
# calculation, file creation imports
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math
from scipy import stats
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() # not sure if this one is actually needed, review required
from forex_python.converter import CurrencyRates
import xlsxwriter as xlsx
import os

#email related imports
import ssl
from email.message import EmailMessage
import smtplib
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

### getting the SnP 
import bs4 as bs
import pickle

Global Functions:

In [2]:
# this is how much money to invest
def portfolioIput():
    global portfolioSize
    canContinue = False
    portfolioSize = input('Enter the size of your portfolio (£)')
    while not canContinue:
        try:
            portfolioSize = int(portfolioSize)
            print('portfolioSize: ',portfolioSize)
            canContinue = True
        except:
            portfolioSize = input('Enter the size of your portfolio (£) (must be numeric)') 
#portfolioIput()

# this is how many stocks to spread the investment over
def stockLimitInput():
    global stockLimit
    canContinue = False
    stockLimit = input('How many stocks do you wish to invest in?')
    while not canContinue:
        try:
            stockLimit = int(stockLimit)
            print('stockLimit: ',stockLimit)
            canContinue = True
        except:
            stockLimit = input('How many stocks do you wish to invest in? (needs to be an integer)') 
#stockLimitInput()    

# this is depreciated
def exchange(amount, currency):
    try:  
        rate = CurrencyRates().get_rate('GBP', currency)
    except:
        rate = 1
    return amount * rate
exchange(123, 'USD')

def makeTodaysFolder():
    global today
    today = datetime.today().strftime('%Y-%m-%d')
    try:
        current = os.getcwd()
        os.mkdir(current+ f'/{today}')
    except:
        pass 
#makeTodaysFolder()

def emailWithAttachment(subject, email_receiver, email_receiver_name,  filename):
    # Define email sender and receiver
    email_sender = 'beniswinner19@gmail.com'
    email_password = 'jcum jpdc uadb ebbz'

    hello = f"Hi {email_receiver_name}, \n\n"
    body = """
    Here are the findings of this months finance calculations, this is not financial advice

    This was an automated email sent via Python, please see 'About & Audit' tab on excel attachment.
    """
    emailText = hello + body

    
    message = MIMEMultipart()
    message['From'] = email_sender
    message['To'] = email_receiver
    message['Subject'] = subject
    message.attach(MIMEText(emailText, "plain"))
    
    #filename = "./2023-12-05/ADBE_2023-09-06_2023-12-05.jpg"
    
    # Open PDF file in binary mode
    
    # We assume that the file is in the directory where you run your Python script from
    with open(filename, "rb") as attachment:
        # The content type "application/octet-stream" means that a MIME attachment is a binary file
        part = MIMEBase("application", "octet-stream")
        part.set_payload(attachment.read())
    
    # Encode to base64
    encoders.encode_base64(part)
    
    # Add header 
    part.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )
    
    # Add attachment to your message and convert it to string
    message.attach(part)
    #text = em.as_string()

    # Add SSL (layer of security)
    context = ssl.create_default_context()

    # Log in and send the email
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(email_sender, email_password)
        smtp.sendmail(email_sender, email_receiver, message.as_string())

# getting current SNP500 
def get_sp500_tickers():
    global wikiLink
    wikiLink = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    resp = requests.get(wikiLink)
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    global wikiTableRaw
    wikiTableRaw = soup.find('table', {'class': 'wikitable sortable sticky-header'})
    tickers = []
    wikiCols = [title.text.strip() for title in wikiTableRaw.find('tr').findAll('th')] # array of all headings
    wikiCols.append('Link')
    wikiTable = pd.DataFrame(columns=wikiCols)

    for row in wikiTableRaw.findAll('tr')[1:]:
        tmpArr = []
        for colNum, col in enumerate(wikiCols[0:len(wikiCols) - 1]):
            tmpArr.append(row.findAll('td')[colNum].text.strip())
        link = row.find('a', {"class":"external text"}).get("href")
        tmpArr.append(link)
        tmpSeries = pd.Series(tmpArr, index=wikiCols)
        wikiTable.loc[len(wikiTable)] = tmpSeries

    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)

    return wikiTable
##get_sp500_tickers()

Getting the Synbols from the csv:

In [3]:
stockLimitInput() # this sets var stockLimit

stockLimit:  10


In [4]:
portfolioIput() # this to set var portfolioSize50

portfolioSize:  100


In [5]:
#df = pd.read_csv("sp_500_stocks.csv")
#df = pd.read_csv("ftse_250_stocks.csv")

df = get_sp500_tickers()
df

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Link
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,0000066740,1902,https://www.nyse.com/quote/XNYS:MMM
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,0000091142,1916,https://www.nyse.com/quote/XNYS:AOS
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,0000001800,1888,https://www.nyse.com/quote/XNYS:ABT
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888),https://www.nyse.com/quote/XNYS:ABBV
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989,https://www.nyse.com/quote/XNYS:ACN
...,...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,0001524472,2011,https://www.nyse.com/quote/XNYS:XYL
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,0001041061,1997,https://www.nyse.com/quote/XNYS:YUM
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,0000877212,1969,https://www.nasdaq.com/market-activity/stocks/...
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,0001136869,1927,https://www.nyse.com/quote/XNYS:ZBH


Getting the data from yFinance based on the Ticker Symbols

In [ ]:
columns = [
    'Ticker', 
    'Company Name',
    'Price', 
    'Currency',
    'Link',
    'Number of Shares to Buy',
    'One-Year Price Return', 
    'One-Year Price Percentile',
    'Six-Month Price Retun',
    'Six-Month Price Percentile',
    'Three-Month Price Retun',
    'Three-Month Price Percentile',
    'One-Month Price Retun',
    'One-Month Price Percentile',
    'Average Percentile'
]
finalDf = pd.DataFrame(columns=columns)
naDf = pd.DataFrame(columns=columns)

#length = df.shape[0]
length = df.shape[0] # set to 30 for testing...
#length = 5
tickerArr = []
#tmpSeries

for i in range(length):
    tmpSym = df.iloc[i,0]
    tickerArr.append(tmpSym)
stringAgg = ' '.join(tickerArr)

oneMonthAgo = (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')
threeMonthsAgo = (datetime.now() - timedelta(days=90)).strftime('%Y-%m-%d')
sixMonthsAgo = (datetime.now() - timedelta(days=180)).strftime('%Y-%m-%d')

timeFrames = [oneMonthAgo, threeMonthsAgo, sixMonthsAgo]

# example of getting historical data
multi = yf.Tickers(stringAgg)
#multi.tickers['A'].history(start=oneMonthAgo).iloc[0]['Open']
#multi.tickers['A'].history(start=threeMonthsAgo).iloc[0]['Open']
#multi.tickers['A'].history(start=sixMonthsAgo).iloc[0]['Open']


for i in range(length):
    tmpSym = df.iloc[i,0]
    try:
        #dict[tmpSym] = tmpChange
        tmpTic  = multi.tickers[tmpSym]
        curPrice = tmpTic.basic_info.last_price
        tmpSeries = pd.Series([
                        tmpSym, 
                        df.loc[i,'Security'],
                        curPrice,
                        tmpTic.basic_info.currency,
                        df.loc[i,'Link'],
                        'N/A',
                        tmpTic.basic_info.year_change,
                        'X',
                        (curPrice / tmpTic.history(start=sixMonthsAgo).iloc[0]['Open']) - 1,
                        'X',
                        (curPrice / tmpTic.history(start=threeMonthsAgo).iloc[0]['Open']) - 1,
                        'X',
                        (curPrice / tmpTic.history(start=oneMonthAgo).iloc[0]['Open']) - 1,
                        'X',
                        'X'
                    ],
                        index=columns
                    )
        finalDf.loc[len(finalDf)] = tmpSeries
        #print(tmpSeries)
        

    except:
        tmpSeries = pd.Series([
                        tmpSym, 
                        'Price', 
                        'Company Name',
                        'Currency',
                        'Link',
                        'Number of Shares to Buy',
                        'One-Year Price Return',
                        'One-Year Price Percentile',
                        'Six-Month Price Retun',
                        'Six-Month Price Percentile',
                        'Three-Month Price Retun',
                        'Three-Month Price Percentile',
                        'One-Month Price Retun',
                        'One-Month Price Percentile',
                        'Average Percentile'
                    ],
                        index=columns
                    )
        naDf.loc[len(finalDf)] = tmpSeries

finalDf

BF.B: No price data found, symbol may be delisted (1d 2024-06-14 -> 2024-12-11)


Adding In Percentile Data:

In [ ]:
percentileColumnsX = [column for column in columns if 'Percentile' in column.split(' ')] 
percentileColumnsX

percentileColumns = [
    {
        "return": "One-Year Price Return",
        "percentile": "One-Year Price Percentile"
    },
    {
        "return": "Six-Month Price Retun",
        "percentile": "Six-Month Price Percentile"
    },
    {
        "return": "Three-Month Price Retun",
        "percentile": "Three-Month Price Percentile"
    },
    {
        "return": "One-Month Price Retun",
        "percentile": "One-Month Price Percentile"
    }
]

finalDf.loc[3, 'One-Year Price Percentile'] = 10
finalDf

# Using scipy.stats to work out percentiles 
for row in finalDf.index:
    tmpTotalPercentage = 0
    for col in percentileColumns:
        tmpPercentil = stats.percentileofscore(finalDf[col["return"]], finalDf.loc[row , col["return"]])
        finalDf.loc[row , col["percentile"]] = tmpPercentil
        tmpTotalPercentage += tmpPercentil
    finalDf.loc[row ,"Average Percentile"] = tmpTotalPercentage / len(percentileColumns)

finalDf


,Ticker,Company Name,Price,Currency,Link,Number of Shares to Buy,One-Year Price Return,One-Year Price Percentile,Six-Month Price Retun,Six-Month Price Percentile,Three-Month Price Retun,Three-Month Price Percentile,One-Month Price Retun,One-Month Price Percentile,Average Percentile
0,MMM,3M,126.408501,USD,https://www.nyse.com/quote/XNYS:MMM,N/A,0.706239,86.427146,0.397691,94.610778,-0.00711,26.347305,-0.078857,6.986028,53.592814
1,AOS,A. O. Smith,77.07,USD,https://www.nyse.com/quote/XNYS:AOS,N/A,0.118924,25.548902,-0.066831,15.768463,-0.084501,9.780439,-0.103525,3.792415,13.722555
2,ABT,Abbott Laboratories,114.949997,USD,https://www.nyse.com/quote/XNYS:ABT,N/A,0.240051,40.718563,0.081148,42.51497,0.148143,80.439122,0.014688,58.083832,55.439122
3,ABBV,AbbVie,189.485001,USD,https://www.nyse.com/quote/XNYS:ABBV,N/A,0.306129,51.896208,0.207799,72.055888,0.037808,44.111776,-0.015474,36.127745,51.047904
4,ACN,Accenture,361.48999,USD,https://www.nyse.com/quote/XNYS:ACN,N/A,0.247192,41.716567,0.187238,64.870259,0.104367,67.265469,0.083014,90.818363,66.167665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,XYL,Xylem Inc.,130.634995,USD,https://www.nyse.com/quote/XNYS:XYL,N/A,0.473108,68.662675,-0.013778,24.550898,-0.072656,11.776447,-0.023144,30.339321,33.832335
497,YUM,Yum! Brands,133.800003,USD,https://www.nyse.com/quote/XNYS:YUM,N/A,0.121347,25.948104,-0.058277,17.165669,0.045966,47.50499,0.0,47.105788,34.431138
498,ZBRA,Zebra Technologies,363.630005,USD,https://www.nasdaq.com/market-activity/stocks/...,N/A,0.767563,90.01996,0.218395,74.451098,0.110728,69.660679,-0.002743,44.91018,69.760479
499,ZBH,Zimmer Biomet,102.815201,USD,https://www.nyse.com/quote/XNYS:ZBH,N/A,0.008534,12.774451,-0.138901,7.784431,-0.07507,11.576846,-0.049679,16.566866,12.175649


Filtering and Ordering:

In [ ]:
finalDfFiltered = finalDf.loc[(finalDf['One-Year Price Return'] != 'N/A') & (finalDf['One-Year Price Return'])]
finalDfFilteredOrdered = finalDfFiltered.sort_values('Average Percentile', ascending = False)

finalDfFilteredOrderedLimited = finalDfFilteredOrdered[:stockLimit]
finalDfFilteredOrderedLimited.reset_index(inplace=True, drop=True)

finalDfFilteredOrderedLimited50 = finalDfFilteredOrdered[:50]
finalDfFilteredOrderedLimited50.reset_index(inplace=True, drop=True)

finalDfFilteredOrderedLimited

,Ticker,Company Name,Price,Currency,Link,Number of Shares to Buy,One-Year Price Return,One-Year Price Percentile,Six-Month Price Retun,Six-Month Price Percentile,Three-Month Price Retun,Three-Month Price Percentile,One-Month Price Retun,One-Month Price Percentile,Average Percentile
0,PLTR,Palantir Technologies,44.764999,USD,https://www.nyse.com/quote/XNYS:PLTR,N/A,1.848921,99.600798,0.967692,100.0,0.624274,99.600798,0.213472,99.800399,99.750499
1,GEV,GE Vernova,296.649994,USD,https://www.nyse.com/quote/XNYS:GEV,N/A,1.091255,96.806387,0.931567,99.800399,0.701755,99.800399,0.153831,98.003992,98.602794
2,NVDA,Nvidia,143.815002,USD,https://www.nasdaq.com/market-activity/stocks/...,N/A,2.481873,99.800399,0.642093,99.001996,0.265083,96.007984,0.178618,99.001996,98.453094
3,UAL,United Airlines Holdings,75.68,USD,https://www.nasdaq.com/market-activity/stocks/ual,N/A,1.133734,97.40519,0.433333,96.407186,0.597636,99.401198,0.412467,100.0,98.303393
4,VST,Vistra Corp.,125.5,USD,https://www.nyse.com/quote/XNYS:VST,N/A,2.817021,100.0,0.720442,99.401198,0.748415,100.0,0.099142,92.814371,98.053892
5,AXON,Axon Enterprise,444.869995,USD,https://www.nasdaq.com/market-activity/stocks/...,N/A,1.166104,97.804391,0.443259,96.806387,0.42404,98.602794,0.127995,96.007984,97.305389
6,RCL,Royal Caribbean Group,201.169998,USD,https://www.nyse.com/quote/XNYS:RCL,N/A,1.449928,99.201597,0.430072,96.207585,0.302585,97.205589,0.124546,95.808383,97.105788
7,IRM,Iron Mountain,129.279999,USD,https://www.nyse.com/quote/XNYS:IRM,N/A,1.134361,97.60479,0.680028,99.201597,0.316677,97.60479,0.081208,90.618762,96.257485
8,TRGP,Targa Resources,168.369995,USD,https://www.nyse.com/quote/XNYS:TRGP,N/A,0.966748,94.810379,0.450623,97.205589,0.280084,96.806387,0.076122,89.021956,94.461078
9,CEG,Constellation Energy,266.952515,USD,https://www.nasdaq.com/market-activity/stocks/ceg,N/A,1.346522,98.802395,0.421529,96.007984,0.519169,99.201597,0.049177,80.638723,93.662675


Calculate the number of Shares to buy based on portfolio size

In [ ]:
curLen = len(finalDfFilteredOrderedLimited.index) # incase there is a differnece between stockLimit and the len of df ??
positionSize = float(portfolioSize) / curLen # this is the amount we have to spend on each stock
for i in range(curLen):
    print(finalDfFilteredOrderedLimited.loc[i, 'Currency'])
    positionSizeConverted = exchange(positionSize, finalDfFilteredOrderedLimited.loc[i, 'Currency'])
    tmpAmount = positionSizeConverted / finalDfFilteredOrderedLimited.loc[i, 'Price']
    finalDfFilteredOrderedLimited.loc[i, 'Currency']
    finalDfFilteredOrderedLimited.loc[i, 'Number of Shares to Buy'] = tmpAmount
finalDfFilteredOrderedLimited # this will return fractional shares, add math.floor when a baller with a huge portfolio 

USD
USD
USD
USD
USD
USD
USD
USD
USD
USD


,Ticker,Company Name,Price,Currency,Link,Number of Shares to Buy,One-Year Price Return,One-Year Price Percentile,Six-Month Price Retun,Six-Month Price Percentile,Three-Month Price Retun,Three-Month Price Percentile,One-Month Price Retun,One-Month Price Percentile,Average Percentile
0,PLTR,Palantir Technologies,44.764999,USD,https://www.nyse.com/quote/XNYS:PLTR,22.338881,1.848921,99.600798,0.967692,100.0,0.624274,99.600798,0.213472,99.800399,99.750499
1,GEV,GE Vernova,296.649994,USD,https://www.nyse.com/quote/XNYS:GEV,3.370976,1.091255,96.806387,0.931567,99.800399,0.701755,99.800399,0.153831,98.003992,98.602794
2,NVDA,Nvidia,143.815002,USD,https://www.nasdaq.com/market-activity/stocks/...,6.953377,2.481873,99.800399,0.642093,99.001996,0.265083,96.007984,0.178618,99.001996,98.453094
3,UAL,United Airlines Holdings,75.68,USD,https://www.nasdaq.com/market-activity/stocks/ual,13.213531,1.133734,97.40519,0.433333,96.407186,0.597636,99.401198,0.412467,100.0,98.303393
4,VST,Vistra Corp.,125.5,USD,https://www.nyse.com/quote/XNYS:VST,7.968127,2.817021,100.0,0.720442,99.401198,0.748415,100.0,0.099142,92.814371,98.053892
5,AXON,Axon Enterprise,444.869995,USD,https://www.nasdaq.com/market-activity/stocks/...,2.247848,1.166104,97.804391,0.443259,96.806387,0.42404,98.602794,0.127995,96.007984,97.305389
6,RCL,Royal Caribbean Group,201.169998,USD,https://www.nyse.com/quote/XNYS:RCL,4.97092,1.449928,99.201597,0.430072,96.207585,0.302585,97.205589,0.124546,95.808383,97.105788
7,IRM,Iron Mountain,129.279999,USD,https://www.nyse.com/quote/XNYS:IRM,7.735149,1.134361,97.60479,0.680028,99.201597,0.316677,97.60479,0.081208,90.618762,96.257485
8,TRGP,Targa Resources,168.369995,USD,https://www.nyse.com/quote/XNYS:TRGP,5.939301,0.966748,94.810379,0.450623,97.205589,0.280084,96.806387,0.076122,89.021956,94.461078
9,CEG,Constellation Energy,266.952515,USD,https://www.nasdaq.com/market-activity/stocks/ceg,3.745985,1.346522,98.802395,0.421529,96.007984,0.519169,99.201597,0.049177,80.638723,93.662675


**xlsxwriter section:**

Creates xlsx file with attachments and emails results as an attachment

In [ ]:
makeTodaysFolder() # also creates global variable of today
fileName = f"./{today}/stock_report_{today}.xlsx"
writer = pd.ExcelWriter(fileName , engine="xlsxwriter")

# creating the top 50 sheet
top50sheetName = 'Top 50'
finalDfFilteredOrderedLimited50.to_excel(writer, sheet_name=top50sheetName)
worksheet = writer.sheets[top50sheetName]
for colnum, column in enumerate(finalDfFilteredOrderedLimited50.columns):
    len(list(column))
    worksheet.set_column(colnum + 1, colnum + 1, len(list(column)) + 1)



# creating the individual stock sheets
for ticker in finalDfFilteredOrderedLimited['Ticker']:
    # iterating throuhg the top 
    mainDf = finalDfFilteredOrderedLimited
    mainDf.loc[mainDf['Ticker'] == ticker].to_excel(writer, sheet_name=ticker)  # Default position, cell A1.
    worksheet = writer.sheets[ticker]

    # setting column widths of the sheet
    for colnum, column in enumerate(mainDf.columns):
        len(list(column))
        worksheet.set_column(colnum + 1, colnum + 1, len(list(column)) + 1)

    worksheet.set_column(0,0,3)

        #link  = str(mainDf.loc[mainDf.Ticker == ticker, 'Link'][0])
        #print(link)
        #worksheet.write(3, 0, link)

    
    for index, date in enumerate(timeFrames):
        # creating, saving and inserting the files into the sheet
        plt.figure(figsize=(10,4))
        yf.Ticker(ticker).history(start=date)['Open'].plot()
        plt.title(f'{ticker} - starting {date}')
        plt.legend([ticker])
        plt.ylabel(yf.Ticker(ticker).basic_info['currency'])
        imageFileName = f"{today}/{ticker}_{date}_{today}.jpg"
        plt.savefig(imageFileName)
        worksheet.insert_image("A" + str(4 + 17 * index), imageFileName, {"x_scale": 0.75, "y_scale": 0.75})
        #plt.show()


#creating the About / Audit
helpAuditSheetName = 'About & Audit'
worksheet = writer.book.add_worksheet(helpAuditSheetName)
worksheet.set_column(0,0,100)

autitMessages = [
    F'This is not financial advice',
    f'Nuber of stocks selected to invest in: {stockLimit}',
    f'Selected Portfolio Size (£): {portfolioSize}',
    f'Tickers evalueted via: {wikiLink}',
    f'Stock data added via the yFinance python library',
    f'This was run on: {datetime.now().strftime("%d/%m/%Y %H:%M:%S")}'
    
]

for index, message in enumerate(autitMessages):
    worksheet.write(index,0,message)


writer.close()
print('Workbook is awaiting')

PermissionError: [Errno 13] Permission denied: './2024-10-25/stock_report_2024-10-25.xlsx'

In [ ]:
emailSubject = f'{today} - Quantitative Momentum Report'
mailingCols = ['Name', 'Email', 'Active']
mailingListData = [
    ['Ben', 'ben.hunt19@hotmail.com', True]
]
#building the dataframe to store the data, does't need to be a DataFrame but could be useful for expansion
mailingDf = pd.DataFrame(columns=mailingCols)
for j in range(len(mailingListData)):
    mailingDf.loc[len(mailingDf)] = pd.Series(mailingListData[j], index=mailingCols)

for i in range(len(mailingDf)):
    if mailingDf.loc[i]['Active']:
        emailWithAttachment(emailSubject, mailingDf.loc[i]['Email'], mailingDf.loc[i]['Name'], fileName)